In [1]:
%pip install pandas -q
%pip install matplotlib -q
%pip install kagglehub -q
%pip install tensorflow -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import kagglehub
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AvgPool2D, GlobalAveragePooling2D, Dropout 
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.applications.resnet50 import preprocess_input
from tensorflow.keras.utils import image_dataset_from_directory

RANDOM_SEED = 42
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

2025-06-29 21:05:17.885827: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-29 21:05:18.217812: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751220318.340552     817 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751220318.375251     817 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751220318.657388     817 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
# загрузим данные
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")


In [4]:
# преднастройка формаирование датасета
settings_loading_data = dict(
    labels='inferred',
    label_mode='binary',
    color_mode='rgb', # если использовать предобученную ч/б модель, то изменить этот параметр на 'grayscale'
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    interpolation='nearest', # Попробовать: bicubic (лучшая, но тяжёлая) lanczos3/5 (вроде норм)
    batch_size=BATCH_SIZE,
    seed=RANDOM_SEED
)

In [5]:
# сформируем датасеты
df_train = image_dataset_from_directory(
    path + '/chest_xray/train',
    shuffle=True,
    **settings_loading_data
)

df_test = image_dataset_from_directory(
    path + '/chest_xray/test',
    shuffle=False,
    **settings_loading_data
)

df_valid = image_dataset_from_directory(
    path + '/chest_xray/val',
    shuffle=False,
    **settings_loading_data
)

Found 5216 files belonging to 2 classes.


I0000 00:00:1751220322.794203     817 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5529 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.9


Found 624 files belonging to 2 classes.
Found 16 files belonging to 2 classes.


In [6]:
# вывод 9 фотографий из датасета для примера
class_names = df_train.class_names
print("Классы:", class_names)

# for images, labels in df_train.take(1):
#     plt.figure(figsize=(10, 10))
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8").squeeze(), cmap='gray') 
#         plt.title(f"Class: {class_names[int(labels[i].numpy().item())]}")
#         plt.axis("off")
#     plt.show()

Классы: ['NORMAL', 'PNEUMONIA']


In [7]:
# проверим баланс классов
# TODO - сделать попроще это ячейку
labels = []
for _, label_batch in df_train:
    labels.extend(label_batch.numpy())

normal_count = np.sum(np.array(labels) == 0)
pneumonia_count = np.sum(np.array(labels) == 1)

print(f"Класс NORMAL (0): {normal_count} изображений")
print(f"Класс PNEUMONIA (1): {pneumonia_count} изображений")

Класс NORMAL (0): 1341 изображений
Класс PNEUMONIA (1): 3875 изображений


2025-06-29 21:05:26.818883: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
# настроим аугментацию
# 1. будем поворачивать фотографии, на 10 градусов в разные стороны
# 2. смещаем фотографии по вертикали и горизонатли на 10 процентов
# 3. зумим фотографии на 20 процентов
data_augmentation_pipeline = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(factor=10/360.0, fill_mode="nearest", interpolation="bilinear"),
    tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1, fill_mode="nearest", interpolation="bilinear"),
    tf.keras.layers.RandomZoom(height_factor=0.2, width_factor=0.2, fill_mode="nearest", interpolation="bilinear")
])

In [9]:
#применим аугментацию
def augment_and_preprocess_train(image, label):
    image_augmented = data_augmentation_pipeline(image, training=True)

    image_float_0_1 = tf.image.convert_image_dtype(image_augmented, tf.float32)
    brightness_factor = tf.random.uniform(shape=[], minval=0.8, maxval=1.2)
    image_brightened = image_float_0_1 * brightness_factor
    image_brightened = tf.clip_by_value(image_brightened, 0.0, 1.0)

    image_for_resnet_preprocess = image_brightened * 255.0
    image_preprocessed = preprocess_input(image_for_resnet_preprocess) 

    return image_preprocessed, label

def preprocess_val_test(image, label):
    image = tf.cast(image, tf.float32) # Просто переводим в float32
    image_preprocessed = preprocess_input(image) # И применяем ту же предобработку ResNet
    return image_preprocessed, label


df_train_processed = df_train.map(augment_and_preprocess_train, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
df_test_processed = df_test.map(preprocess_val_test, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
df_valid_processed = df_valid.map(preprocess_val_test, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

In [10]:
# создадим модель
tf.keras.backend.set_image_data_format('channels_last')
backbone = ResNet50(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), include_top=False)
model = Sequential()
model.add(backbone)
model.add(GlobalAveragePooling2D())
model.add(Dense(units=1, activation='sigmoid'))

optimizer = Adam(learning_rate=2e-4) 
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[
        'accuracy',       
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc')
    ])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         2,049 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,589,761 (89.99 MB)

 Trainable params: 23,536,641 (89.79 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [11]:
# %pip uninstall tensorflow # если установлен обычный
# %pip install tensorflow==2.10.0
# %pip install tensorflow-directml-plugin

In [12]:
import tensorflow as tf

print("Версия TensorFlow:", tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Ограничиваем рост памяти GPU по мере необходимости
        # Это нужно делать в самом начале, до инициализации GPU
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Физических GPU,", len(logical_gpus), "Логических GPU доступно")
        print("GPU доступен:", tf.test.is_gpu_available(cuda_only=True)) # Для TF < 2.5
        print("GPU доступен (новый метод):", len(tf.config.list_physical_devices('GPU')) > 0)
    except RuntimeError as e:
        # Рост памяти должен быть установлен до инициализации GPU
        print(e)
else:
    print("GPU не найдены.")

Версия TensorFlow: 2.19.0
Physical devices cannot be modified after being initialized


In [13]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

# Очень полезные коллбэки
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6, verbose=1)
# Сохраняет лучшую версию модели
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
# Останавливает обучение, если нет улучшений, чтобы не тратить время и не переобучаться
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Не забудьте использовать обработанные датасеты!
model.fit(
    df_train_processed,
    validation_data=df_valid_processed,
    epochs=10, # Можно поставить побольше, EarlyStopping сам остановит
    callbacks=[reduce_lr, model_checkpoint, early_stopping]
)

Epoch 1/10


I0000 00:00:1751220345.653455    2329 service.cc:152] XLA service 0x7efdc0002480 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751220345.653480    2329 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Ti, Compute Capability 8.9
2025-06-29 21:05:46.798501: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1751220350.484059    2329 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1751220369.077528    2329 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.8090 - auc: 0.8335 - loss: 0.4511 - precision: 0.8534 - recall: 0.8930

163/163 ━━━━━━━━━━━━━━━━━━━━ 75s 206ms/step - accuracy: 0.8091 - auc: 0.8337 - loss: 0.4508 - precision: 0.8535 - recall: 0.8931 - val_accuracy: 0.5000 - val_auc: 0.4609 - val_loss: 1.5197 - val_precision: 0.5000 - val_recall: 1.0000 - learning_rate: 2.0000e-04
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step - accuracy: 0.8424 - auc: 0.8849 - loss: 0.3737 - precision: 0.8790 - recall: 0.9102

163/163 ━━━━━━━━━━━━━━━━━━━━ 31s 186ms/step - accuracy: 0.8424 - auc: 0.8849 - loss: 0.3735 - precision: 0.8791 - recall: 0.9102 - val_accuracy: 0.5000 - val_auc: 0.5391 - val_loss: 1.4200 - val_precision: 0.5000 - val_recall: 1.0000 - learning_rate: 2.0000e-04
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.8508 - auc: 0.8988 - loss: 0.3420 - precision: 0.8899 - recall: 0.9089

163/163 ━━━━━━━━━━━━━━━━━━━━ 34s 204ms/step - accuracy: 0.8509 - auc: 0.8988 - loss: 0.3420 - precision: 0.8900 - recall: 0.9090 - val_accuracy: 0.5000 - val_auc: 0.3125 - val_loss: 0.8818 - val_precision: 0.5000 - val_recall: 1.0000 - learning_rate: 2.0000e-04
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 32s 191ms/step - accuracy: 0.8587 - auc: 0.9105 - loss: 0.3221 - precision: 0.8939 - recall: 0.9160 - val_accuracy: 0.5000 - val_auc: 0.2969 - val_loss: 1.5540 - val_precision: 0.5000 - val_recall: 1.0000 - learning_rate: 2.0000e-04
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step - accuracy: 0.8655 - auc: 0.9107 - loss: 0.3239 - precision: 0.8961 - recall: 0.9239
Epoch 5: ReduceLROnPlateau reducing learning rate to 3.9999998989515007e-05.
163/163 ━━━━━━━━━━━━━━━━━━━━ 32s 196ms/step - accuracy: 0.8656 - auc: 0.9107 - loss: 0.3238 - precision: 0.8961 - recall: 0.9239 - val_accuracy: 0.5000 - val_auc: 0.6250 - val_loss: 3.8588 - val_precision: 0.5000 - val_recall: 1.0000 - learning_rate